In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import numpy as np
from time import sleep
import requests
from tqdm.notebook import tqdm
import time
import random
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re

In [2]:
main_link = 'https://comicbookrealm.com/series/5871/0/image-comics-the-walking-dead'

In [48]:
def start_chrome(headless=True, driver_path='./'):
    # use selenium webdriver so we can scrape the javascript elements
    # chromedriver should be in the path or it should be provided. version should correspond to the chrome version installed
    # Link for versions: https://chromedriver.chromium.org/downloads
    if headless:
        from selenium.webdriver.chrome.options import Options
        opts = Options()
        opts.add_argument('--blink-settings=imagesEnabled=false')
        opts.add_argument(" --headless")
        chrome_driver = "chromedriver"# Instantiate a webdriver
        browser = webdriver.Chrome(options=opts, executable_path=driver_path+chrome_driver)# Load the HTML page
    else:
        from selenium.webdriver.chrome.options import Options
        opts = Options()
        opts.add_argument('--blink-settings=imagesEnabled=false')
        chrome_driver = "chromedriver"
        browser = webdriver.Chrome(options=opts, executable_path=driver_path+chrome_driver)

    
    return browser

def issue_scraper(browser, issue_link, issue_nr, cover_link, title, title_link, sleep_time=0.8):
    
    data_dict = {'title': title, 
                 'title_link': title_link, 
                 'issue_link': issue_link,
                 'cover_link': cover_link}
    
    browser.get(issue_link)
    # WebDriverWait(browser, wait_time).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'page_1')))
    time.sleep(sleep_time + random.random())
    try:
        # click on accept cookies and privacy aggreements
        browser.find_elements_by_xpath("//button[@class=' css-47sehv']")[0].click()
        time.sleep(1) # wait a bit for the page to load
        browser.find_elements_by_xpath("//a[@class='cc_btn cc_btn_accept_all']")[0].click()
        time.sleep(1) # wait a bit for the page to load
    except IndexError:
        pass
    
    soup_file = browser.page_source
    soup = BeautifulSoup(soup_file, 'html.parser')
    issue_data = (soup.find('div', id='comic-details-content')
                     .find('table', class_='auto-table')
                     .find_all('td', class_='data')
                 )
    issue_labels = (soup.find('div', id='comic-details-content')
                        .find('table', class_='auto-table')
                        .find_all('td', class_='label')
                   )
    labels = ['issue', 'cover_date', 'cover_price', 'current_value',
        'searched', 'owned', 'pages', 'rating', 'ISBN-UPC', 
       'est_print_run', 'variant_of', 'preview']

    for dat, label, lab in zip(issue_data, issue_labels, labels):
        data_dict[lab] = dat.text.strip()
        if label.text == 'Current Value:':
            data_dict['hist_prices_link'] = dat.find('a').get('href')
        if dat.find('img') is not None and label.text == 'Rating:':
            rating = dat.find('img').get('title')
            rating = re.findall("\d+\.\d+|\d+", rating)
            data_dict['rating'] = float(rating[0])
            data_dict['rating_count'] = int(rating[1])
            
    data_dict['synopsis'] = soup.find('div', id='synopsis').text.strip()
    # get info from contributors tab
    browser.find_elements_by_xpath("//li[@ref='contributors']")[0].click()
    time.sleep(sleep_time)
    
    soup_file = browser.page_source
    soup = BeautifulSoup(soup_file, 'html.parser')
    creators = soup.find('div', id='contributors').find_all('li')
    
    if len(creators) == 1 and creators[0].text == 'There have been no contributors assigned to this issue':
        names = 'no contributors assigned'
        jobs = 'no contributors assigned'
    else:
        names = []
        jobs = []
        for li in creators:
            contrib = li.text.strip()
            names.append(contrib.split('\n', 1)[0])
            j = contrib.split('\n', 1)[1]
            jobs.append(re.sub('\s*', '', j))

    
    data_dict['contributors_names'] = str(names)
    data_dict['contributors_roles'] = str(jobs)
    
    # get info from characters tab
    browser.find_elements_by_xpath("//li[@ref='characters']")[0].click()
    time.sleep(sleep_time)
    
    soup_file = browser.page_source
    soup = BeautifulSoup(soup_file, 'html.parser')
    
    characters = soup.find('div', id='characters').find_all('li')
    
    if len(characters) == 1 and characters[0].text == 'There have been no characters assigned to this issue':
        character_data = 'no characters assigned'
    
    else:
        character_data = []
        for li in characters:
            name = li.find('a').text.strip()
            photo_link = li.find('img').get('src')
            extra_info = li.find('span').text.strip()
            if '(' not in extra_info:
                extra_info = ''

            event = li.find('span', class_='d').text.strip()
            character_data.append([name, photo_link, extra_info, event])
    
    data_dict['characters'] = str(character_data)
    
    return pd.DataFrame(data_dict, index=[0])
    
    
            

In [49]:
# df = pd.read_csv('all_issues_list_data.csv', index_col=0)

In [50]:
# browser = start_chrome(headless=True)

In [51]:
# # test the scraper function
# index = 10000
# issue_link = df.issue_link[index]
# issue_nr = df.issue_nr[index]
# cover_link = df.cover_link[index]
# title = df.title[index]
# title_link = df.title_link[index]
# sleep_time=1

# site_link = 'https://comicbookrealm.com'

# data = issue_scraper(browser, site_link+issue_link, issue_nr, cover_link, title, title_link, sleep_time)

### Make the scraper loop

In [29]:
df = pd.read_csv('all_issues_data.csv', index_col=0)

In [30]:
browser = start_chrome(headless=True)

In [52]:
site_link = 'https://comicbookrealm.com'
sleep_time = 1
df1 = df.iloc[31:32]

iterates = tqdm(zip(range(df1.shape[0]), 
                    df1.issue_link, 
                    df1.issue_nr, 
                    df1.cover_link, 
                    df1.title,
                    df1.title_link,
                   )
               )

frames_list = []

for i, issue_link, issue_nr, cover_link, title, title_link in iterates:
    
    iterates.set_description(f"Processing: {title}: {issue_nr} - {i + 1} / {df1.shape[0]}")
    
    try:
        frame = issue_scraper(browser, site_link+issue_link, issue_nr, cover_link, title, title_link, sleep_time)
    
    except AttributeError:
        time.sleep(2)
        try:
            frame = issue_scraper(browser, site_link+issue_link, issue_nr, cover_link, title, title_link, sleep_time)
        except AttributeError:
            continue
    
        
    
    frames_list.append(frame)
    
    if i%10000 == 0 and i > 0:
        temp = pd.concat(frames_list, axis=0)
        temp.to_csv(f'issues_data_{i}.csv')

browser.quit()
dff = pd.concat(frames_list, axis=0).reset_index(drop=True)   
dff.to_csv('single_issue_data_part1.csv')

0it [00:00, ?it/s]

This comic was rated 0.00 stars by 0 members ['0.00', '0']


In [37]:
re.findall("\d+\.\d+|\d+", 'This comic was rated 0.00 stars by 0 members')

['0.00', '0']

In [20]:
dff = pd.concat(frames_list, axis=0).reset_index(drop=True)

In [35]:
issue_link


'/series/43774/379663/dc-comics-52-tpb-tpb-1'

In [22]:
browser.quit()

In [23]:
dff.to_csv('single_issue_data_part1.csv')

## Minimal Example

In [12]:
browser = start_chrome(headless=False)

In [13]:
browser.get(main_link)
# WebDriverWait(browser, wait_time).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'page_1')))
time.sleep(1) 

In [62]:
soup_file = browser.page_source
soup = BeautifulSoup(soup_file)

In [74]:
issue_data = soup.find('div', id='comic-details-content').find('table', class_='auto-table').find_all('td', class_='data')

In [75]:
issue_labels = soup.find('div', id='comic-details-content').find('table', class_='auto-table').find_all('td', class_='label')

In [115]:
labels = ['issue', 'cover_date', 'cover_price', 'current_value',
        'searched', 'owned', 'pages', 'rating', 'ISBN-UPC', 
       'est_print_run', 'variant_of', 'preview']

data_dict = {}
for dat, label, lab in zip(issue_data, issue_labels, labels):
    data_dict[lab] = dat.text.strip()
    if label.text == 'Current Value:':
        data_dict['hist_prices_link'] = dat.find('a').get('href')
    if dat.find('img') is not None and label.text != 'Current Value:':
        print(lab)
        rating = dat.find('img').get('title')
        rating = re.findall("\d+\.\d+|\d+", rating)
        data_dict['rating'] = float(rating[0])
        data_dict['rating_count'] = int(rating[1])
        
        


rating


In [121]:
data_dict['synopsis'] = soup.find('div', id='synopsis').text.strip()

In [126]:
browser.find_elements_by_xpath("//button[@class=' css-47sehv']")[0].click()
time.sleep(0.2)

In [127]:
browser.find_elements_by_xpath("//a[@class='cc_btn cc_btn_accept_all']")[0].click()
time.sleep(0.2)

In [128]:
# get info from contributors tab
browser.find_elements_by_xpath("//li[@ref='contributors']")[0].click()
time.sleep(0.2)`

In [130]:
soup_file = browser.page_source
soup = BeautifulSoup(soup_file)

In [170]:
creators = soup.find('div', id='contributors').find_all('li')

In [175]:
names = []
jobs = []
for li in creators:
    contrib = li.text.strip()
    names.append(contrib.split('\n', 1)[0])
    j = contrib.split('\n', 1)[1]
    jobs.append(re.sub('\s*', '', j))

contributors = [names, jobs]

In [176]:
contributors

[['Robert Kirkman', 'Tony Moore', 'Cliff Rathburn'],
 ['Scripts-Letters', 'Pencils-CoverArt-Inks', 'Colors']]

In [178]:
# get info from characters tab
browser.find_elements_by_xpath("//li[@ref='characters']")[0].click()
time.sleep(0.2)

In [179]:
soup_file = browser.page_source
soup = BeautifulSoup(soup_file)

In [181]:
characters = soup.find('div', id='characters').find_all('li')

In [191]:
if len(characters) == 1 and characters[0].text == 'There have been no characters assigned to this issue':
    characters_data == 'no characters assigned'
else:
    character_data = []
    for li in characters:
        name = li.find('a').text.strip()
        photo_link = li.find('img').get('src')
        extra_info = li.find('span').text.strip()
        if '(' not in extra_info:
            extra_info = ''

        event = li.find('span', class_='d').text.strip()
        character_data.append([name, photo_link, extra_info, event])

In [192]:
character_data

[['Duane Jones',
  '/character-photos/character_27481_t.jpg',
  '',
  'First appearance of...'],
 ['Morgan Jones',
  '/character-photos/character_27480_t.jpg',
  '',
  'First appearance of...'],
 ['Rick Grimes',
  '/character-photos/character_25360_t.jpg',
  '(Rick Grimes)',
  'First appearance of...'],
 ['Shane Walsh',
  '/character-photos/character_27484_t.jpg',
  '(Shane Walsh)',
  'First appearance of...'],
 ['Zombies', '/character-photos/character_12938_t.jpg', '', '']]